https://stackabuse.com/minimax-and-alpha-beta-pruning-in-python/

In [2]:
import time
import numpy as np

In [3]:
row = 7
columns = 6
inarow = 4

Let's set the fundation of the game

In [60]:
class Game:
    def __init__(self):
        self.initialize_game()
    
    def initialize_game(self):
        self.current_state = [
            ['X', 'O', 'X', 'X', 'O', 'O'],
            ['O', 'X', 'O', 'X', 'O', 'O'],
            ['X', 'X', 'O', 'X', 'O', 'O'],
            ['O', 'O', 'X', 'O', 'X', 'X'],
            ['O', 'X', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.'],
            ['.', '.', '.', '.', '.', '.']
        ]
        # Player X always plays first
        self.player_turn = 'X'
        
    def draw_board(self):
        for i in range(0, row):
            for j in range(0, columns):
                print('{}|'.format(self.current_state[i][j]), end=" ")
            print()
        print()
    
    
    ### CHECK THE GAME ###
    
    
    def is_valid(self, px, py):
        
        """Function that defines wether a move is valid or not"""
        
        if px < 0 or px > 6 or py < 0 or py > 7:
            return False
        elif self.current_state[px][py] != '.':
            return False
        elif px != 6 and px+1 == '.': # 0 is upside and 6 is downside
            return False
        else:
            return True
        
    def is_end(self):
        
        """Check if there is x inarow and if so return the winner"""
        
        # Vertical win
        win_x = ['X', 'X', 'X', 'X']
        win_o = ['O', 'O', 'O', 'O']
        for j in range(0,columns):
            state = []
            for i in range(0,row):
                state.append(self.current_state[i][j])
            for i in range(0,row):
                if win_x == state[0+i:inarow+i]:
                    return 'X'
                elif win_o == state[0+i:inarow+i]:
                    return 'O'
            

        # Horizontal win
        for i in range(0, row):
            for j in range(0,columns):
                if win_x == self.current_state[i][0+j:j+inarow]:
                    return 'X'
                elif win_o == self.current_state[i][0+j:j+inarow]:
                    return 'O'

        # Diagonal win
        max_col = len(self.current_state[0])
        max_row = len(self.current_state)
        cols = [[] for _ in range(max_col)]
        rows = [[] for _ in range(max_row)]
        fdiag = [[] for _ in range(max_row + max_col - 1)]
        bdiag = [[] for _ in range(len(fdiag))]
        min_bdiag = -max_row + 1

        for x in range(max_col):
            for y in range(max_row):
                cols[x].append(self.current_state[y][x])
                rows[y].append(self.current_state[y][x])
                fdiag[x+y].append(self.current_state[y][x])
                bdiag[x-y-min_bdiag].append(self.current_state[y][x])

        for i in bdiag:
            if len(i) >= inarow:
                for j in range(0,row):
                    if win_x == i[0+j:j+inarow]:
                        #return (i[0+j:j+inarow])
                        return 'X'
                    elif win_o == i[0+j:j+inarow]:
                        return 'O'
        for i in fdiag:
            if len(i) >= inarow:
                for j in range(0,row):
                    if win_x == i[0+j:j+inarow]:
                        return 'X'
                    elif win_o == i[0+j:j+inarow]:
                        return 'O'

        # Is whole board full?
        for i in range(0, row):
            for j in range(0, columns):
                # There's an empty field, we continue the game
                if (self.current_state[i][j] == '.'):
                    return None

        # It's a tie!
        return '.'
    
    
    ### MINIMAX ALGO ### 
    
    
    # Player 'O' is max, in this case Algo
    def max_alpha_beta(self, alpha, beta):
        maxv = -2
        px = None
        py = None

        result = self.is_end()

        if result == 'X':
            return (-1, 0, 0)
        elif result == 'O':
            return (1, 0, 0)
        elif result == '.':
            return (0, 0, 0)

        for i in range(0, row):
            for j in range(0, columns):
                if self.current_state[i][j] == '.':
                    self.current_state[i][j] = 'O'
                    (m, min_i, in_j) = self.min_alpha_beta(alpha, beta)
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    self.current_state[i][j] = '.'

                    # Basique rule of Alpha/Beta pruning
                    if maxv >= beta:
                        return (maxv, px, py)

                    if maxv > alpha:
                        alpha = maxv

        return (maxv, px, py)
    
    # Player 'X' is min, in this case human
    def min_alpha_beta(self, alpha, beta):

        minv = 2

        qx = None
        qy = None

        result = self.is_end()

        if result == 'X':
            return (-1, 0, 0)
        elif result == 'O':
            return (1, 0, 0)
        elif result == '.':
            return (0, 0, 0)

        for i in range(0, row):
            for j in range(0, columns):
                if self.current_state[i][j] == '.':
                    self.current_state[i][j] = 'X'
                    (m, max_i, max_j) = self.max_alpha_beta(alpha, beta)
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.current_state[i][j] = '.'

                    # Basique rule of Alpha/Beta pruning
                    if minv <= alpha:
                        return (minv, qx, qy)

                    if minv < beta:
                        beta = minv

        return (minv, qx, qy)

    
    ### PLAY THE GAME ###
    
    
    def play(self):
        while True:
            self.draw_board()
            self.result = self.is_end()

            # Printing the appropriate message if the game has ended
            if self.result != None:
                if self.result == 'X':
                    print('The winner is X!')
                elif self.result == 'O':
                    print('The winner is O!')
                elif self.result == '.':
                    print("It's a tie!")
                elif self.result == 'bdiag':
                    print('bdiag')
                elif self.result == 'fdiag':
                    print('fdiag')
                elif self.result == 'horizontal':
                    print('horizontal')
                elif self.result == 'vertical':
                    print('vertical')

                self.initialize_game()
                return

            # If it's player's turn
            if self.player_turn == 'X':

                while True:

                    start = time.time()
                    (m, qx, qy) = self.min_alpha_beta(-2,2)
                    end = time.time()
                    print('Evaluation time: {}s'.format(round(end - start, 7)))
                    print('Recommended move: X = {}, Y = {}'.format(qx, qy))

                    px = int(input('Insert the X coordinate: '))
                    py = int(input('Insert the Y coordinate: '))

                    (qx, qy) = (px, py)

                    if self.is_valid(px, py):
                        # We modify the game state
                        self.current_state[px][py] = 'X'
                        self.player_turn = 'O'
                        break
                    else:
                        print('The move is not valid! Try again.')

            # If it's algo's turn
            else:
                (m, px, py) = self.max_alpha_beta(-2,2)
                # algo modify the game state
                print('Move is:', px, 'x', py, 'y')
                #print('The move is not valid!')
                self.current_state[px][py] = 'O'
                self.player_turn = 'X'

In [61]:
def main():
    g = Game()
    g.play()

if __name__ == "__main__":
    main()

X| O| X| X| O| O| 
O| X| O| X| O| O| 
X| X| O| X| O| O| 
O| O| X| O| X| X| 
O| X| .| .| .| .| 
.| .| .| .| .| .| 
.| .| .| .| .| .| 

Evaluation time: 1432.7712085s
Recommended move: X = 5, Y = 0
Insert the X coordinate: 5
Insert the Y coordinate: 0
X| O| X| X| O| O| 
O| X| O| X| O| O| 
X| X| O| X| O| O| 
O| O| X| O| X| X| 
O| X| .| .| .| .| 
X| .| .| .| .| .| 
.| .| .| .| .| .| 

The winner is X!


160 secondes pour calculer le 28 eme coups (27 coups sur le terrain) <br/>
1432 secondes pour calculer le 27 eme coups (26 coups sur le terrain)